In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install -i https://test.pypi.org/simple/ my_krml_25076922==2024.0.3.4

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import os

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data/processed'

sales_data = pd.read_csv(f'{path}/df_pred.csv')

In [5]:
sales_data.head()

,item_id,store_id,date,total_sales,day_of_week,month,year
0,HOBBIES_1_001,CA_1,2011-01-29,0.0,5,1,2011
1,HOBBIES_1_002,CA_1,2011-01-29,0.0,5,1,2011
2,HOBBIES_1_003,CA_1,2011-01-29,0.0,5,1,2011
3,HOBBIES_1_004,CA_1,2011-01-29,0.0,5,1,2011
4,HOBBIES_1_005,CA_1,2011-01-29,0.0,5,1,2011


In [8]:
sales_data['item_id'].value_counts()

item_id
HOBBIES_1_001      15450
FOODS_2_210        15450
FOODS_2_201        15450
FOODS_2_202        15450
FOODS_2_203        15450
                   ...  
HOUSEHOLD_1_464    15450
HOUSEHOLD_1_465    15450
HOUSEHOLD_1_466    15450
HOUSEHOLD_1_467    15450
FOODS_3_827        15450
Name: count, Length: 3049, dtype: int64

In [9]:
df_cleaned = sales_data.copy()

In [10]:
columns_to_drop = ['date']

df_cleaned = df_cleaned.drop(columns=columns_to_drop)

df_cleaned.head()

,item_id,store_id,total_sales,day_of_week,month,year
0,HOBBIES_1_001,CA_1,0.0,5,1,2011
1,HOBBIES_1_002,CA_1,0.0,5,1,2011
2,HOBBIES_1_003,CA_1,0.0,5,1,2011
3,HOBBIES_1_004,CA_1,0.0,5,1,2011
4,HOBBIES_1_005,CA_1,0.0,5,1,2011


In [11]:
from my_krml_25076922.features.data_processing import encode_categorical_columns

# List of categorical columns to encode
categorical_columns = ['store_id', 'item_id']

# Apply label encoding to the categorical columns
df_cleaned = encode_categorical_columns(df_cleaned, categorical_columns)

In [12]:
df_cleaned.head()

,item_id,store_id,total_sales,day_of_week,month,year
0,1437,0,0.0,5,1,2011
1,1438,0,0.0,5,1,2011
2,1439,0,0.0,5,1,2011
3,1440,0,0.0,5,1,2011
4,1441,0,0.0,5,1,2011


In [13]:
from sklearn.preprocessing import StandardScaler

# Define the features to scale
features_to_scale = ['total_sales', 'day_of_week', 'month', 'year']

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler and transform the selected features
df_cleaned[features_to_scale] = scaler.fit_transform(df_cleaned[features_to_scale])

In [14]:
df_cleaned.head()

,item_id,store_id,total_sales,day_of_week,month,year
0,1437,0,-0.330511,0.993873,-1.539218,-1.371861
1,1438,0,-0.330511,0.993873,-1.539218,-1.371861
2,1439,0,-0.330511,0.993873,-1.539218,-1.371861
3,1440,0,-0.330511,0.993873,-1.539218,-1.371861
4,1441,0,-0.330511,0.993873,-1.539218,-1.371861


In [16]:
from joblib import dump

dump(scaler, '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/scaler.joblib')

['/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/scaler.joblib']

In [18]:
from my_krml_25076922.data.sets import pop_target

features, target = pop_target(df_cleaned, 'total_sales')

In [19]:
from my_krml_25076922.data.sets import split_sets_random

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(features, target, test_ratio=0.2)

In [20]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(28264230, 5)
(9421410, 5)
(9421410, 5)


In [21]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(28264230,)
(9421410,)
(9421410,)


In [24]:
from my_krml_25076922.data.sets import save_sets

save_sets(X_train, y_train, X_val, y_val, X_test, y_test, path='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data/processed')

In [25]:
from my_krml_25076922.models.null import NullRegressor

base_model = NullRegressor()

In [26]:
y_base = base_model.fit_predict(y_train)

In [28]:
from my_krml_25076922.models.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')

RMSE Training: 0.9980232715094798
MAE Training: 0.477523215846306
